In [17]:
import requests
from bs4 import BeautifulSoup
import os
import re
import time
from urllib.parse import urljoin, urlparse

def sanitize_filename(filename):
    """파일명으로 사용할 수 없는 문자를 제거하고 적절한 길이로 조정합니다."""
    # 파일명으로 사용할 수 없는 문자를 제거
    filename = re.sub(r'[\\/*?:"<>|]', '', filename)
    # 공백이나 특수문자 처리
    filename = re.sub(r'\s+', '_', filename)
    # 파일명 길이 제한 (Windows는 255자)
    if len(filename) > 240:
        filename = filename[:240]
    return filename

def crawl_faq(start_url, output_dir="faq_data"):
    """
    FAQ 사이트의 모든 페이지를 크롤링하여 질문과 답변을 추출하고 텍스트 파일로 저장합니다.
    
    Args:
        start_url (str): 크롤링을 시작할 URL
        output_dir (str): 결과물을 저장할 디렉토리
    """
    # 출력 디렉토리 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 이미 방문한 URL을 추적
    visited_urls = set()
    # 크롤링할 URL 큐
    queue = [start_url]
    # 베이스 URL 추출 (같은 도메인인지 확인하기 위함)
    base_domain = urlparse(start_url).netloc
    
    # 저장된 FAQ 수를 계산
    faq_count = 0
    
    while queue:
        current_url = queue.pop(0)
        
        # 이미 방문했거나 같은 도메인이 아니면 건너뜀
        if current_url in visited_urls or urlparse(current_url).netloc != base_domain:
            continue
        
        # 방문 표시
        visited_urls.add(current_url)
        
        try:
            print(f"크롤링 중: {current_url}")
            response = requests.get(current_url, headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
            })
            response.raise_for_status()  # 오류 확인
            
            # 페이지 파싱
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # FAQ 항목 찾기
            faq_items = soup.select('.faq_content')
            
            for item in faq_items:
                # 질문과 답변 추출 방식은 사이트 구조에 따라 다를 수 있음
                # 여기서는 일반적인 구조를 가정
                question_element = item.find_previous(lambda tag: tag.name in ['h2', 'h3', 'h4', 'dt', 'strong'] or 
                                                   (tag.name == 'div' and 'question' in tag.get('class', [])))
                
                # 질문 요소를 찾지 못한 경우
                if not question_element:
                    continue
                
                question_text = question_element.get_text(strip=True)
                answer_text = item.get_text(strip=True)
                
                # 질문이나 답변이 비어있지 않은 경우에만 저장
                if question_text and answer_text:
                    # 파일명으로 사용할 수 없는 문자 처리
                    safe_filename = sanitize_filename(question_text)
                    
                    # 중복 파일명 방지
                    file_path = os.path.join(output_dir, f"{safe_filename}.txt")
                    counter = 1
                    while os.path.exists(file_path):
                        file_path = os.path.join(output_dir, f"{safe_filename}_{counter}.txt")
                        counter += 1
                    
                    # 파일에 답변 저장
                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(answer_text)
                    
                    faq_count += 1
                    print(f"저장됨: {safe_filename}")
            
            # 현재 페이지에서 다른 링크 찾기
            links = soup.find_all('a', href=True)
            for link in links:
                href = link['href']
                # 상대 URL을 절대 URL로 변환
                absolute_url = urljoin(current_url, href)
                
                # 같은 도메인인 경우만 큐에 추가
                if urlparse(absolute_url).netloc == base_domain and absolute_url not in visited_urls:
                    queue.append(absolute_url)
            
            # 서버에 부담을 주지 않기 위한 딜레이
            time.sleep(1)
        
        except Exception as e:
            print(f"오류 발생: {current_url} - {str(e)}")
    
    print(f"\n크롤링 완료! 총 {faq_count}개의 FAQ를 저장했습니다.")
    print(f"저장 경로: {os.path.abspath(output_dir)}")

if __name__ == "__main__":
    # FAQ 사이트의 시작 URL을 입력하세요
    start_url = input("FAQ 사이트 URL을 입력하세요: ")
    output_dir = input("결과물을 저장할 디렉토리를 입력하세요 (기본값: faq_data): ") or "faq_data"
    
    crawl_faq(start_url, output_dir)

크롤링 중: https://www.i-sh.co.kr/centgk/brd/m_44/view.do
크롤링 중: https://www.i-sh.co.kr/centgk/brd/m_44/view.do#container
크롤링 중: https://www.i-sh.co.kr/main/lay2/program/S1T2C3/www/m_593/login/join_step1Page.do
크롤링 중: https://www.i-sh.co.kr/centgk/m_33/site/sitemap.do
크롤링 중: https://www.i-sh.co.kr/centgk/brd/m_44/view.do#familysite
크롤링 중: https://www.i-sh.co.kr/main/index.do


KeyboardInterrupt: 

In [10]:
pip install selenium webdriver-manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


0개의 게시글을 찾았습니다.
